### lncRNAs quantified? realive to background
1. construct backgorund gtf
2. map reads with rnaseqc
3. compare to see if lncRNAs are quantified

In [ ]:
import pandas as pd
import seaborn as sns
import pyranges as pr
import numpy as np
import bisect

from tqdm import tqdm
tqdm.pandas()

#### create background gtf

In [217]:
exons = pd.read_csv("/home/klawren/oak/gtex/data/other_references/nongenic_null/all_exons.bed", sep='\t', header=None, names=['chr', 'start', 'end', 'exon_id', 'gene_id'])
exons['length'] = exons['end'] - exons['start']
exons = exons[~exons['chr'].isin(['chrM'])]
background = pd.read_csv("/home/klawren/oak/gtex/data/other_references/nongenic_null/background_regions.bed", sep='\t', header=None, names=['chr', 'start', 'end'])
background = background[background['chr'].isin(exons['chr'].unique())]
background = background.sort_values(['chr', 'start'])

In [218]:
def get_closest_exon(exon, background_starts, background_ends):
    exon_length = exon['end'] - exon['start']
    idx_right = bisect.bisect_left(background_starts, exon['start'])
    # catch index too far left
    while idx_right < len(background_starts) and background_starts[idx_right] < exon['start']:
        idx_right += 1

    idx_left = idx_right - 1
    found = False
    max_iter = len(background_starts)  # Safety limit
    iter_count = 0
    fits_right=False
    fits_left=False

    while not found and iter_count < max_iter:
        iter_count += 1
        # Try right side
        if idx_right < len(background_starts):
            right_start, right_end = background_starts[idx_right], background_ends[idx_right]
            if right_end - right_start >= exon_length:
                fits_right=True
        
        # Try left side
        if idx_left >= 0:
            left_start, left_end = background_starts[idx_left], background_ends[idx_left]
            if left_end - left_start >= exon_length:
                fits_left=True
            
        if fits_right and fits_left:
            # if both fit, compare distances
            right_distance = right_start - exon['end']
            left_distance = exon['start'] - left_end
            if right_distance < left_distance:
                return([right_start, right_start + exon_length])
            else:
                return([left_end-exon_length, left_end])
        elif fits_right:
            return([right_start, right_start + exon_length])
        elif fits_left:
            return([left_end-exon_length, left_end])
        
        idx_right += 1
        idx_left -= 1
        fits_right=False
        fits_left=False


In [219]:
null_exons = []
for chrom, background_chrom in background.groupby('chr'):
    null_exons_chr = []
    print(chrom)
    background_starts = background_chrom['start'].values
    background_ends = background_chrom['end'].values
    exon_chrom = exons[exons['chr'] == chrom]
    print(len(exon_chrom))
    for idx, exon in tqdm(exon_chrom.iterrows(), total=len(exon_chrom)):
        null_exons_chr.append(get_closest_exon(exon, background_starts, background_ends))
    null_exons_chr = pd.DataFrame(null_exons_chr, columns=['start', 'end'])
    null_exons_chr['exon_id'] = exon_chrom['exon_id'].values
    null_exons_chr['gene_id'] = exon_chrom['gene_id'].values
    null_exons_chr['chr'] = chrom
    null_exons.append(null_exons_chr)

chr1
40634


100%|███████████████████████████████████████████████████████████████████████████████████| 40634/40634 [00:04<00:00, 8700.44it/s]


chr10
18178


100%|██████████████████████████████████████████████████████████████████████████████████| 18178/18178 [00:01<00:00, 10069.43it/s]


chr11
22260


...

 ... (more hidden) ...


chrX
13991


 ... (more hidden) ...


chrY
2887


 ... (more hidden) ...


In [220]:
null_exons_all_chrs = pd.concat(null_exons)

In [221]:
null_exons_all_chrs

,start,end,exon_id,gene_id,chr
0,10929,11020,ENSG00000290825.2_1; exon_number 1,ENSG00000290825.2,chr1
1,10758,11020,ENSG00000290825.2_2; exon_number 2,ENSG00000290825.2,chr1
2,10945,11020,ENSG00000290825.2_3; exon_number 3,ENSG00000290825.2,chr1
3,10855,11020,ENSG00000290825.2_4; exon_number 4,ENSG00000290825.2,chr1
4,10899,11020,ENSG00000290825.2_5; exon_number 5,ENSG00000290825.2,chr1
...,...,...,...,...,...
2882,57212948,57213025,ENSG00000292371.1_1; exon_number 1,ENSG00000292371.1,chrY
2883,57213731,57213755,ENSG00000310542.1_4; exon_number 4,ENSG00000310542.1,chrY
2884,57214082,57214100,ENSG00000310542.1_3; exon_number 3,ENSG00000310542.1,chrY
2885,57214830,57215163,ENSG00000310542.1_2; exon_number 2,ENSG00000310542.1,chrY


In [225]:
null_exons_all_chrs[['chr', 'start', 'end', 'exon_id', 'gene_id']].to_csv('/home/klawren/oak/gtex/data/other_references/nongenic_null/matched_nongenic_null.bed', index=False, header=None, sep = '\t')

In [226]:
gencode_v47_path = "/home/klawren/oak/gtex/data/realign_references/gencode.v47.genes.gtf"
gencode_v47=pr.read_gtf(gencode_v47_path)
gencode_v47_df = gencode_v47.as_df()

In [232]:
gencode_v47_df = gencode_v47_df[~(gencode_v47_df['Chromosome'] == 'chrM')]

"each gene have a single transcript whose id matches the parent gene id" from rnaseqc

In [ ]:
# fix the null name to be compatible
null_exons_all_chrs['exon_id'] = null_exons_all_chrs['exon_id'].str.split(';').str[0]

In [261]:
len(null_exons_all_chrs)

432330

In [262]:
len(gencode_v47_df)

581512

In [ ]:
# merge in the null information
null_exons_gtf = pd.merge(gencode_v47_df[gencode_v47_df['Feature'] == 'exon'].reset_index(), null_exons_all_chrs, on='exon_id', suffixes=['', '_null'])
# combine into the main gtf based on index 
null_full_gtf = pd.merge(gencode_v47_df.reset_index(), null_exons_gtf[['index', 'start', 'end']], on='index', how='left', suffixes=['', '_null'])

# add gene starts and ends as bounds 
gene_starts = null_full_gtf.groupby('gene_id').agg({'start':'min', 'end':'max'})
null_full_gtf = pd.merge(null_full_gtf, gene_starts, on='gene_id', how='left', suffixes=['', '_gene'])

# replace the start and end with the null version
null_full_gtf['Start'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['start'], null_full_gtf['start_gene'])
null_full_gtf['End'] = np.where(null_full_gtf['Feature'] == 'exon', null_full_gtf['end'], null_full_gtf['end_gene'])

In [ ]:
# Function to convert the DataFrame back to GTF format
def df_to_gtf(df, output_file):
    with open(output_file, 'w') as f:
        f.write("##description: nongenic nulls matched to exons from annotation of the human genome (GRCh38), version 47 (Ensembl 113)\n##provider: Kate Lawrence\n##contact: klawren@stanford.edu\n##format: gtf\n##date: 2025-03-21")
        for index, row in df.iterrows():
            attributes = f'gene_id "{row["gene_id"]}"; transcript_id "{row["transcript_id"]}"; exon_id "{row["exon_id"]}";'
            gtf_line = f'{row["Chromosome"]}\t{row["Source"]}\t{row["Feature"]}\t{row["Start"]}\t{row["End"]}\t{row["Score"]}\t{row["Strand"]}\t{row["Frame"]}\t{attributes}\n'
            f.write(gtf_line)

In [260]:
df_to_gtf(null_full_gtf, '/home/klawren/oak/gtex/data/other_references/nongenic_null/matched_nongenic_null.v47.gtf')

In [263]:
len(null_full_gtf)

581512